## 공공데이터 API 기반 크롤링 

In [2]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 2s (4,103 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletyp

In [2]:
import os
import sys
import urllib.request
import datetime
import time
import json
import pandas as pd

ServiceKey="F%2BtgdbNOVzV5OtqJjdwNNZRRlVfwF6vxOkdJ4EZJ7TYF1PawSea3PHrEo0JGfszlUuwUAkc%2BnKEXrp4uFGiBiA%3D%3D"

#[CODE 1]
def getRequestUrl(url):    
    req = urllib.request.Request(url)    
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None


#[CODE 2]
def getTourismStatsItem(yyyymm, national_code, ed_cd):    
    service_url = "http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList"
    parameters = "?_type=json&serviceKey=" + ServiceKey   #인증키
    parameters += "&YM=" + yyyymm
    parameters += "&NAT_CD=" + national_code
    parameters += "&ED_CD=" + ed_cd
    url = service_url + parameters
    
    retData = getRequestUrl(url)   #[CODE 1]
    
    if (retData == None):
        return None
    else:
         return json.loads(retData)

#[CODE 3]
def getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear):
    jsonResult = []
    result = []
    natName=''
    dataEND = "{0}{1:0>2}".format(str(nEndYear), str(12)) #데이터 끝 초기화
    isDataEnd = 0 #데이터 끝 확인용 flag 초기화    
    
    for year in range(nStartYear, nEndYear+1):        
        for month in range(1, 13):
            if(isDataEnd == 1): break #데이터 끝 flag 설정되어있으면 작업 중지.
            yyyymm = "{0}{1:0>2}".format(str(year), str(month))            
            jsonData = getTourismStatsItem(yyyymm, nat_cd, ed_cd) #[CODE 2]
            
            if (jsonData['response']['header']['resultMsg'] == 'OK'):               
                # 입력된 범위까지 수집하지 않았지만, 더이상 제공되는 데이터가 없는 마지막 항목인 경우 -------------------
                if jsonData['response']['body']['items'] == '': 
                    isDataEnd = 1 #데이터 끝 flag 설정
                    dataEND = "{0}{1:0>2}".format(str(year), str(month-1))
                    print("데이터 없음.... \n 제공되는 통계 데이터는 %s년 %s월까지입니다." %(str(year), str(month-1)))                    
                    break                
                #jsonData를 출력하여 확인......................................................
                print (json.dumps(jsonData, indent=4,sort_keys=True, ensure_ascii=False))          
                natName = jsonData['response']['body']['items']['item']['natKorNm']
                natName = natName.replace(' ', '')
                num = jsonData['response']['body']['items']['item']['num']
                ed = jsonData['response']['body']['items']['item']['ed']
                print('[ %s_%s : %s ]' %(natName, yyyymm, num))
                print('----------------------------------------------------------------------')                
                jsonResult.append({'nat_name': natName, 'nat_cd': nat_cd,'yyyymm': yyyymm, 'visit_cnt': num})
                result.append([natName, nat_cd, yyyymm, num])
                
    return (jsonResult, result, natName, ed, dataEND)

#[CODE 0]
def main():
    jsonResult = []
    result = []
    natName=''
    print("<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>")
    nat_cd = input('국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) : ')
    nStartYear =int(input('데이터를 몇 년부터 수집할까요? : '))
    nEndYear = int(input('데이터를 몇 년까지 수집할까요? : '))
    ed_cd = "E"     #E : 방한외래관광객, D : 해외 출국
    
    jsonResult, result, natName, ed, dataEND =getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear) #[CODE 3]

    if (natName=='') : #URL 요청은 성공하였지만, 데이터 제공이 안된 경우
        print('데이터가 전달되지 않았습니다. 공공데이터포털의 서비스 상태를 확인하기 바랍니다.')
    else:
        #파일저장 1 : json 파일       
        with open('./%s_%s_%d_%s.json' % (natName, ed, nStartYear, dataEND), 'w', encoding='utf8') as outfile:
            jsonFile  = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)
            outfile.write(jsonFile)
        #파일저장 2 : csv 파일   
        columns = ["입국자국가", "국가코드", "입국연월", "입국자 수"]
        result_df = pd.DataFrame(result, columns = columns)
        result_df.to_csv('./%s_%s_%d_%s.csv' % (natName, ed, nStartYear, dataEND),index=False, encoding='utf8')
    
if __name__ == '__main__':
    main()

<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>
국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) : 275
데이터를 몇 년부터 수집할까요? : 2020
데이터를 몇 년까지 수집할까요? : 2021
[2022-05-22 13:34:32.121230] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 275,
                    "natKorNm": "미  국",
                    "num": 67255,
                    "rnum": 1,
                    "ym": 202001
                }
            },
            "numOfRows": 10,
            "pageNo": 1,
            "totalCount": 1
        },
        "header": {
            "resultCode": "0000",
            "resultMsg": "OK"
        }
    }
}
[ 미국_202001 : 67255 ]
----------------------------------------------------------------------
[2022-05-22 13:34:32.277356] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객

## 라이브러리를 이용한 데이터 수집

BeautifulSoup 정적 웹 페이지를 크롤링

Selenium 동적 웹 페이지를 크롤링 

In [9]:
from bs4 import BeautifulSoup

html = '<h1 id="title">한빛출판네트워크</h1><div class="top"><ulclass="menu"><li><a href=http://www.hanbit.co.kr/member/login.htmlclass="login">로그인 </a></li></ul><ul class="brand"><li><a href="http://www. hanbit.co.kr/media/>한빛미디어<li><a href="http://www.hanbit.co.kr/academy/">한빛아카데미</a></li></ul></div>'
soup = BeautifulSoup(html,'html.parser')

print(soup.prettify())

<h1 id="title">
 한빛출판네트워크
</h1>
<div class="top">
 <ulclass="menu">
  <li>
   <a href='http://www.hanbit.co.kr/member/login.htmlclass="login"'>
    로그인
   </a>
  </li>
 </ulclass="menu">
</div>
<ul class="brand">
 <li>
  <a "="" academy="" href="http://www. hanbit.co.kr/media/&gt;한빛미디어&lt;li&gt;&lt;a href=" http:="" www.hanbit.co.kr="">
   한빛아카데미
  </a>
 </li>
</ul>


In [10]:
soup.h1

<h1 id="title">한빛출판네트워크</h1>

In [12]:
tag_div = soup.div
tag_div

<div class="top"><ulclass="menu"><li><a href='http://www.hanbit.co.kr/member/login.htmlclass="login"'>로그인 </a></li></ulclass="menu"></div>

In [13]:
tag_ul = soup.ul
tag_ul

<ul class="brand"><li><a "="" academy="" href="http://www. hanbit.co.kr/media/&gt;한빛미디어&lt;li&gt;&lt;a href=" http:="" www.hanbit.co.kr="">한빛아카데미</a></li></ul>

In [14]:
#find_all 지정된 태그를 모두 파싱

tag_ul_all = soup.find_all('ul')
tag_ul_all

[<ul class="brand"><li><a "="" academy="" href="http://www. hanbit.co.kr/media/&gt;한빛미디어&lt;li&gt;&lt;a href=" http:="" www.hanbit.co.kr="">한빛아카데미</a></li></ul>]

In [15]:
tag_a_all = soup.find_all('a')
tag_a_all

[<a href='http://www.hanbit.co.kr/member/login.htmlclass="login"'>로그인 </a>,
 <a "="" academy="" href="http://www. hanbit.co.kr/media/&gt;한빛미디어&lt;li&gt;&lt;a href=" http:="" www.hanbit.co.kr="">한빛아카데미</a>]

In [18]:
#attrs : 속성 이름과 속성값으로 딕셔너리 구성

tag_div.attrs

{'class': ['top']}

In [20]:
tag_div['class']

['top']

In [21]:
# find(): 속성을 이용하여 특정 태그 파싱
tag_ul_2 = soup.find('ul', attrs = {'class':'brand'})
tag_ul_2

<ul class="brand"><li><a "="" academy="" href="http://www. hanbit.co.kr/media/&gt;한빛미디어&lt;li&gt;&lt;a href=" http:="" www.hanbit.co.kr="">한빛아카데미</a></li></ul>

In [22]:
title = soup.find(id = 'title')
title

<h1 id="title">한빛출판네트워크</h1>

In [23]:
title.string

'한빛출판네트워크'

In [27]:
#select(): 지정한 태그를 모두 파싱하여 리스트 구성 
# 태그#id 속성값/ 태그.class 속성값

li_list = soup.select('div>ul.brand>li')
li_list

[]